In [ ]:
import socket
import time
import os
import sys
import platform
from pathlib import Path
import datetime as datetime
import subprocess
import re

module_path = os.path.abspath(os.path.join('../../wpshelper'))
if module_path not in sys.path:
    sys.path.append(module_path)
from wpshelper import wps_open, wps_open_capture, wps_export_audio, wps_export_pcapng, wps_wireless_devices, wps_find_installations, wps_analyze_capture, wps_get_available_streams_audio, wps_close


In [2]:
from typing import Optional
# run the remote commands on a Linux machine
def run_command(command):
    if command is None:
        command = "ls" # default command 

    # Run the command and capture the output
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)
    stdout, stderr = process.communicate()
    stdout_str = str(stdout)
    # Check for errors
    if process.returncode != 0:
        log_entry = f"run_command: An error occurred while executing the command {command}:\n{stderr}"
    else:
        log_entry = f"run_command: Command output:\n{stdout}"
    print(log_entry)
    return stdout_str

def start_server(wps_executable_path: str) -> Optional[subprocess.Popen]:
    server_program = 'FTSAutoServer.exe'
    auto_server_path = os.path.join(wps_executable_path, server_program)
    if not os.path.exists(auto_server_path):
        raise FileNotFoundError(
            f"Auto server executable not found at {auto_server_path}. "
            "Set --auto-server-path to the correct location."
        )
    process = subprocess.Popen([auto_server_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(2)
    return process

In [3]:

# =======================START OF CONFIGURATION SECTION (USER MUST MODIFY TO MATCH THEIR SETTINGS)================
# See the manual Automation Server Protocol.pdf for more details
TCP_IP = '127.0.0.1'    # The IP address of the interface that the automation server is running on, normally local host 127.0.0.1
TCP_PORT = 22901        # The default port to talk with the automation server

MAX_TO_READ = 1000

# This will capture Bluetooth LE
# This is described in the section "IOParameters – Sodera and X500 of the Automation Server Protocol manual
# IMPORTANT: This is ignored for the X240. For configurations that include one or more X240 devices, the capture 
# technology should be configured by using the appropriate datasource prior to starting capture via automation. 
# Configuration of capture technology and initiating firmware update are not supported via the automation interface. 
capture_technology = "capturetechnology=bredr-off|le-on|2m-on|spectrum-off|wifi-off"

# Only one personality_key should be uncommented and it should be the device that is connected and powered up before
# running the script. Additional personality keys to support other equipment and configurations 
# can be found under the section "Start FTS" in the Automation Server Protocol manual.
#
# Only uncomment one of the personality_key statements below:
# personality_key = "SODERA"
# personality_key = "X240"  
# personality_key = "X500"
# personality_key = "X500e"
personality_key = "VIEW"

# Setup the path to the the location of FTS.exe. Change this to your directory
DEFAULT_WPS_BASE_DIR = r"C:\Program Files (x86)\Teledyne LeCroy Wireless"

# ===================================END OF CONFIGURATION SECTION=====================================

wps_installations = wps_find_installations(base_dir=DEFAULT_WPS_BASE_DIR, show_log=False)
latest_install = (wps_installations or {}).get("latest") or {}

wps_path=latest_install.get("path", "")


# Automatically derive the following data
personality_key = personality_key.strip()  # remove any spaces

wps_executable_path = wps_path + '\\' +  'Executables\\Core'
# verify the path exists
if not Path(wps_executable_path).exists():
    print("WPS Executable path does not exist: " + wps_executable_path)
else:
    print("Using WPS Path: " + wps_executable_path)

Using WPS Path: C:\Program Files (x86)\Teledyne LeCroy Wireless\Wireless Protocol Suite 4.50\Executables\Core


In [5]:
auto_server_process = start_server(wps_executable_path)
if auto_server_process is not None:
    print(f"auto_server_process started")

auto_server_process started


In [6]:

wps_handle = wps_open(tcp_ip=TCP_IP,tcp_port=TCP_PORT,max_to_read = 1000,wps_executable_path=wps_executable_path,personality_key=personality_key)

In [14]:
wps_wireless_devices(wps_handle, action="select", action_parameters={'type':'bluetooth','address':'all','select':'yes'}, show_log=True)
wps_analyze_capture(wps_handle,show_log=True)


wps_wireless_devices: sending: b'Wireless Devices;select;type=bluetooth;address=all;select=yes'
_recv_and_parse: WIRELESS DEVICES;SUCCEEDED;Timestamp=1/14/2026 12:20:44 PM

wps_wireless_devices: WIRELESS DEVICES;SUCCEEDED;Timestamp=1/14/2026 12:20:44 PM



NameError: name 'wps_analyze_capture' is not defined

In [7]:
capture_absolute_filename=r'C:\Users\potto\Documents\data\bluetooth\adafruit_ADV_EXT_IND.cfax'
capture_pcapng_export_filename=r'C:\Users\potto\Documents\data\bluetooth\adafruit_ADV_EXT_IND.pcapng'

In [8]:
wps_open_capture(wps_handle, capture_absolute_filename)

In [ ]:
# Export pcapng


wps_export_pcapng(wps_handle, capture_pcapng_export_filename,'le')
wps_close(wps_handle)

: 

In [ ]:
capture_absolute_filename =  data_input_path + '\\' +  capture_name + ".cfax"
capture_audio_export_filename =  data_output_path + '\\' +  capture_name
wps_open_capture(wps_handle, capture_absolute_filename)
available_streams_audio=wps_get_available_streams_audio(wps_handle)
print(f"Available audio streams: {available_streams_audio}")
wps_export_audio(wps_handle, capture_audio_export_filename, audio_streams="1&2")
wps_close(wps_handle)

In [8]:
wps_close(wps_handle)